In [19]:
import numpy as np
import matplotlib.pyplot as plt
import cmasher as cmr
from unyt import Msun, yr, Angstrom, deg, K, km, s, cm, erg

from synthesizer.grid import Grid
from synthesizer.emission_models.agn.models import DiscIncidentEmission
from synthesizer.particle import BlackHoles

# Set a random number seed to ensure consistent results
np.random.seed(42)

In [20]:
# set style
plt.style.use('../matplotlibrc.txt')

First we need to initialise our `BlackHole` object with the parameters that will be needed to compute spectra.

In [21]:

blackholes = BlackHoles(
    masses=np.array([1e7, 1e8, 1e9])*Msun,
    accretion_rates=np.array([0.2, 1, 2])*Msun/yr,
)

Like other `synthesizer` objects we can get more information using the `print` command.

In [22]:
print(blackholes)

+----------------------------------------------------------------------------------+
|                                    PARTICLES                                     |
+---------------------------+------------------------------------------------------+
| Attribute                 | Value                                                |
+---------------------------+------------------------------------------------------+
| nparticles                | 3                                                    |
+---------------------------+------------------------------------------------------+
| metallicity_floor         | 1.00e-05                                             |
+---------------------------+------------------------------------------------------+
| name                      | 'Black Holes'                                        |
+---------------------------+------------------------------------------------------+
| component_type            | 'BlackHoles'                       

/Users/sw376/Dropbox/Research/projects/synthesizer/src/synthesizer/utils/ascii_table.py:67: FutureWarning: The `particle_photo_fluxes` attribute is deprecated. Use `particle_photo_fnu` instead. Will be removed in v1.0.0
  self.attributes[name] = getattr(obj, name)
/Users/sw376/Dropbox/Research/projects/synthesizer/src/synthesizer/utils/ascii_table.py:67: FutureWarning: The `photo_fluxes` attribute is deprecated. Use `photo_fnu` instead. Will be removed in v1.0.0
  self.attributes[name] = getattr(obj, name)
/Users/sw376/Dropbox/Research/projects/synthesizer/src/synthesizer/utils/ascii_table.py:67: FutureWarning: The `particle_photo_luminosities` attribute is deprecated. Use `particle_photo_lnu` instead. Will be removed in v1.0.0
  self.attributes[name] = getattr(obj, name)


## Generating spectral energy distribution

We can generate spectra by passing a blackhole emission model (e.g. the `UnifiedAGN` model described in [emission_models](emission_models.ipynb)) to the spectra creation method (`get_intinsic_spectra`).

## Open the disc grid

In [23]:

grid_name = 'qsosed-isotropic'
grid_dir = "/Users/sw376/Dropbox/Research/data/synthesizer/grids/"

grid = Grid(
    grid_name=grid_name, 
    grid_dir=grid_dir,
    read_lines=False)

print(grid)

+----------------------------------------------------------------------------------------------------------------+
|                                                      GRID                                                      |
+--------------------------------+-------------------------------------------------------------------------------+
| Attribute                      | Value                                                                         |
+--------------------------------+-------------------------------------------------------------------------------+
| grid_dir                       | '/Users/sw376/Dropbox/Research/data/synthesizer/grids/'                       |
+--------------------------------+-------------------------------------------------------------------------------+
| grid_name                      | 'qsosed-isotropic'                                                            |
+--------------------------------+----------------------------------------------

# Initialise the emission model

In [24]:

emission_model = DiscIncidentEmission(grid)

print(emission_model)


|==== EmissionModel: disc_incident ===|
|-------------------------------------|
|  DISC_INCIDENT (blackhole)          |
|-------------------------------------|
|Extraction model:                    |
|  Grid: qsosed-isotropic             |
|  Extract key: incident              |
|  Escape fraction: None              |
|  Save emission: True                |
|=====================================|


### Generate spectra 

In [25]:
# get the spectra assuming this emission model
blackholes.get_particle_spectra(emission_model)

# shorthand
spectra = blackholes.particle_spectra['disc_incident']



print(spectra.shape)

print(spectra.bolometric_luminosity)
print(blackholes.bolometric_luminosity)

(3, 1999)
[1.13259503e+45 5.66297517e+45 1.13259503e+46] erg/s
[1.13259503e+45 5.66297517e+45 1.13259503e+46] erg/s


/var/folders/1r/dyc06cz53k1g87b17q09by98000ft2/T/ipykernel_13622/544038840.py:2: FutureWarning: get_particle_spectra is now just a wrapper around get_spectra. It will be removed by v1.0.0.
  blackholes.get_particle_spectra(emission_model)


In [26]:
ionising_photon_luminosity = spectra.calculate_ionising_photon_production_rate()

ionising_bolometric_correction = ionising_photon_luminosity/spectra.bolometric_luminosity


print(ionising_bolometric_correction)



[1.01760167e+10 1.07291908e+10 9.92560639e+09] 1/erg


## H-alpha luminosities

In [27]:

# for converting Q to Halpha luminosity from Kennicutt 1998
correction_factor = (1.08E-53 / 7.9E-42) * erg

halpha_luminosity = ionising_photon_luminosity * correction_factor

print(halpha_luminosity)

[1.57561145e+43 8.30631300e+43 1.53683897e+44] erg/s


In [1]:
from unyt import dimensionless